In [1]:
import pandas as pd

In [2]:
df_order_details = pd.read_csv(r'C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/raw/order_details.csv')

df_order_details

,order_detail_id,order_id,product_id,quantity,discount,tax_rate,total_price,profit
0,OD0000001,O202311050000,P140,1,0.26,0.21,1144.77,NaN
1,OD0000002,O202311050000,P134,1,0.23,0.21,79.54,0.93
2,OD0000003,O202311050000,P151,1,0.08,0.21,720.52,NaN
3,OD0000004,O202505200001,P157,5,0.10,0.21,931.37,115.78
4,OD0000005,O202505200001,P106,1,NaN,0.21,1540.05,315.92
...,...,...,...,...,...,...,...,...
10344,OD0010345,O202411074997,P106,4,NaN,0.21,6160.21,1263.68
10345,OD0010346,O202411074997,P154,4,0.26,0.21,584.34,-20.24
10346,OD0010347,O202302174998,P109,4,0.28,0.21,1478.15,133.69
10347,OD0010348,O202302174998,P112,3,0.05,0.21,1211.35,NaN


In [3]:
df_order_details_clean = df_order_details.copy()

# Identify exact duplicates (same order, same product, and same quantity)
duplicate_columns = ['order_id', 'product_id', 'quantity']

# Find duplicates
duplicates = df_order_details_clean[df_order_details_clean.duplicated(subset=duplicate_columns, keep='first')]

print(f"Found {len(duplicates)} exact duplicate records")

# Remove duplicates (keep first occurrence)
df_order_details_clean = df_order_details_clean.drop_duplicates(subset=duplicate_columns, keep='first')

print(f"✓ Removed {len(duplicates)} duplicate records")
print(f"✓ Remaining: {len(df_order_details_clean)} records")

Found 29 exact duplicate records
✓ Removed 29 duplicate records
✓ Remaining: 10320 records


In [4]:
# Replace NULL discount with 0.0
df_order_details_clean['discount'] = df_order_details_clean['discount'].fillna(0.0)

print(f"✓ Filled {(df_order_details['discount'].isna()).sum()} NULL discounts with 0.0")

✓ Filled 1563 NULL discounts with 0.0


In [6]:
df_products_clean = pd.read_csv(r'C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/products_cleaned.csv')

df_products_clean

,product_id,product_name,category,subcategory,unit_price,cost_price,supplier,stock_status,cost_price_imputed
0,P101,Logitech MX Master 3 Mouse,Technology,Computers,1114.91,652.090,Leitz,Low Stock,False
1,P102,IKEA Markus Chair,Technology,Networking,57.35,40.145,Microsoft,Low Stock,True
2,P103,HP Envy 15 Laptop,Accessories,Cables,846.25,613.810,Bose,Low Stock,False
3,P104,Leitz Paper Organizer,Furniture,Storage,424.50,314.000,Samsung,In Stock,False
4,P105,Philips LED Desk Lamp,Home Office,Lighting,241.66,153.250,HP,In Stock,False
5,P106,"Samsung 27"" Monitor",Technology,Peripherals,1272.77,956.850,Sony,Low Stock,False
6,P107,Dell XPS 13 Laptop,Technology,Networking,127.40,90.090,Staedtler,Low Stock,False
7,P108,Microsoft Surface Keyboard,Office Supplies,Paper Products,113.63,84.460,HP,Low Stock,False
8,P109,Logitech C920 Webcam,Technology,Networking,424.17,271.980,Staedtler,Out of Stock,False
9,P110,IKEA Bekant Desk,Office Supplies,Paper Products,407.79,290.350,TP-Link,Low Stock,False


In [8]:
# Create a lookup dictionary for faster access
product_prices = df_products_clean.set_index('product_id')[['unit_price', 'cost_price']].to_dict('index')

print(f"Loaded prices for {len(product_prices)} products")

# Recalculate profit only where it's NULL
def recalc_profit_if_null(row):
    """Only recalculate if profit is NULL"""
    
    # If profit exists, keep it
    if pd.notna(row['profit']):
        return row['profit']
    
    # Get product prices from dictionary
    product_info = product_prices.get(row['product_id'])
    
    if product_info and pd.notna(product_info['cost_price']):
        unit_price = product_info['unit_price']
        cost_price = product_info['cost_price']
        
        # Calculate: revenue - cost
        revenue = unit_price * row['quantity'] * (1 - row['discount'])
        cost = cost_price * row['quantity']
        
        return round(revenue - cost, 2)
    
    return None

# Apply to rows with NULL profit
null_profit_mask = df_order_details_clean['profit'].isna()
print(f"\n📊 Recalculating profit for {null_profit_mask.sum()} NULL values...")

df_order_details_clean.loc[null_profit_mask, 'profit'] = df_order_details_clean[null_profit_mask].apply(
    recalc_profit_if_null, axis=1
)

print(f"✓ NULL profits remaining: {df_order_details_clean['profit'].isna().sum()}")

Loaded prices for 60 products

📊 Recalculating profit for 1191 NULL values...
✓ NULL profits remaining: 0


In [9]:
# Add flag for negative profit (for analysis later)
df_order_details_clean['is_loss_sale'] = df_order_details_clean['profit'] < 0

print(f"✓ Flagged {df_order_details_clean['is_loss_sale'].sum()} negative profit sales")

✓ Flagged 844 negative profit sales


In [10]:
# Save cleaned order_details
df_order_details_clean.to_csv('C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/order_details_cleaned.csv', index=False)
print("✓ Saved: C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/order_details_cleaned.csv")

✓ Saved: C:/Users/Eljan Mammadbayov/OneDrive/Рабочий стол/Python Practice/Euromart Claude/order_details_cleaned.csv
